In [1]:
import sys
import subprocess
from pathlib import Path

repo_root = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()

sys.path.append(repo_root)

In [2]:
import numpy as np

from matplotlib import pyplot as plt

from modules import utils, fitting

import importlib
importlib.reload(utils)
importlib.reload(fitting)

utils.set_plot_configs()

In [3]:
class PhotoelectricData:
    def __init__(self,data_file='23-02-20_364nm.txt'):
        self._read_data(data_file)
        self._analyse_all_bins()

    def _read_data(self,data_file):
        l = []
        Is = []

        with open(data_file,'r') as f:
            for line in f.readlines():
                line_data = line.split()
                
                l.append(float(line_data[0]))
                Is.append([
                    float(x) for x in line_data[1:]
                ])

        self.l = np.array(l)
        self.Is = np.array(Is)

    def _analyse_single_bin(self,I):
        bayes_gaussian = fitting.BayesianGaussian(I)

        return bayes_gaussian.mu, bayes_gaussian.sigma, bayes_gaussian.chisqr
    
    def _analyse_all_bins(self):
        self.I_mu = np.array([])
        self.I_sigma = np.array([])
        self.I_reduced_chisqr = np.array([])

        for T in self.Ts:
            mu, sigma, chisqr = self._analyse_single_bin(T)

            self.I_mu = np.append(self.I_mu,mu)
            self.I_sigma = np.append(self.I_sigma,sigma)
            self.I_reduced_chisqr = np.append(self.I_reduced_chisqr,chisqr/T.shape[0])

In [4]:
photoelectric_data = PhotoelectricData()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (37,) + inhomogeneous part.